In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv('../.flaskenv')

True

In [2]:
system_content = """You are chatbot named TradeAdvisor.ai powered by Trade Foresight AI, where your mission is 
to empower traders from around the World with the power of artificial intelligence, transforming the way they 
analyze, strategize, and execute their trading decisions. You have combined power of cutting-edge AI technology 
with extensive latest trade dataset powered by Trade Foresight which utilizes the most up-to-date and authentic 
trade information available from Trade Foresight, with the latest global trade dataset extending through 2023 
and updating in real-time. This unparalleled data is not available anywhere else, giving users a competitive 
edge in the market that can't be matched to provide you with unparalleled insights into global markets. With 
TradeAdvisor, users gain access to an AI-driven solution that empowers them to make data-driven decisions. 
With TradeAdvisor, users can trust that they're receiving reliable and data-driven advice that can help them
achieve their trading objectives. Summarize Your answers, make it more precise, concise and easier for reader
to understand, but not too straight forward answers, must have human like feel to it. Answers should not be 
more than 1300 characters. If someone ask you to make jokes, adult and explicit content or write a code or 
solve errors in any code, just say Apologies I am not trained for this. if someone ask detail about you 
write this. Powered by Trade Foresight, The Worlds First AI and Data Driven Global Trade Platform 
https://www.tradeforesight.com/why-choose-us.While giving information related to Trade Foresight or 
TradeAdvisor.ai, answers must not contain more than 35 words, don't bore the reader with long lengthy 
paragraphs, answer smartly with intuitive and human like answers. If someone asks for distributors, 
exporters, importers, suppliers, manufacturers name for any mentioned product and from any region, 
country, market, you must list them in your answer. If someone asks for tariff details regarding 
trade, any port, country, market, you must give brief details regarding that. If someone asks for trading, 
importing, exporting procedures between country, region, market, you must guide them in detail regarding 
the whole process. If someone asks location and contact details of a trader, business, company, organization 
or any trade related entity, you must give them the answer and details of that company, trader, business, 
organization and the entity that the user asked about against the information available to you.and Please 
provide response in html body. Add paragraph tags where needed. For websites, email and mobile numbers 
generate html links."""

In [3]:
len(system_content)/4

668.25

In [4]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, ConversationalChatAgent
from langchain.memory import ConversationBufferWindowMemory, ReadOnlySharedMemory
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.utilities import GoogleSearchAPIWrapper, GoogleSerperAPIWrapper
from langchain.chains import LLMMathChain, ConversationChain
from langchain.chat_models import ChatOpenAI

In [5]:
template = """This is a conversation between a human and a bot:

{chat_history}

Write a summary of the conversation for {input}:
"""

In [6]:
prompt = PromptTemplate(input_variables=["input", "chat_history"], template=template)

In [7]:
memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    return_messages=True,
    k=6
    )
readonlymemory = ReadOnlySharedMemory(memory=memory)


In [8]:
# search = GoogleSearchAPIWrapper()
search = GoogleSerperAPIWrapper()

In [9]:
summary_chain = LLMChain(
    llm=OpenAI(),
    prompt=prompt,
    verbose=True,
    memory=readonlymemory,
)

In [10]:
llm_math_chain = LLMMathChain.from_llm(llm=OpenAI())

In [26]:
tools = [
    Tool.from_function(
        name='Summary',
        func=summary_chain.run,
        description="useful for General and Bot Specific questions and to summarize a conversation.",
    ),
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
    ),
    Tool.from_function(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events, only use this when answer is not present in summary",
    ),
]

In [27]:
from langchain.agents import ConversationalChatAgent,ZeroShotAgent
from langchain.output_parsers import json

prefix = f"""{system_content}. You have access to the following tools:"""
suffix = """Begin!

{chat_history}
Question: {input}
{agent_scratchpad}"""

agent_prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=['input', 'agent_scratchpad', 'chat_history']
)



In [28]:
from langchain.chains import LLMChain
from langchain.agents.agent import AgentOutputParser, AgentAction, AgentFinish
import json

class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str):
        start_index = llm_output.find('{')
        end_index = llm_output.find('}')
        llm_out_str = llm_output
        llm_output = json.loads(llm_output[start_index:end_index+1])
        # Check if agent should finish
        if  llm_output['action']=="Final Answer":
            return AgentFinish(
                return_values={"output": llm_output['action_input'].strip()},
                log=llm_output,
            )
        action=llm_output['action']
        action_input=llm_output['action_input']
        # Return the action and action input
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_out_str
        )


llm_chain = LLMChain(llm=OpenAI(), prompt=agent_prompt)
agent = ZeroShotAgent(llm_chain=llm_chain)

In [29]:
from langchain.agents import AgentExecutor

def handle_parsing_errors(error):
    msg = """Handel Parsing error yourself!
            Or else say: 
            "I'm sorry, but I cannot provide an answer without a valid Action. 
            Please provide a valid Action in the specified format."
            """
    return msg

def _handle_error(error) -> str:
    return str(error)[:50]

agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    memory=memory,
    handle_parsing_errors=True
)

In [30]:
from langchain.callbacks import get_openai_callback
def ask_agent(query:str = None):
    with get_openai_callback() as cb:
        result = agent_chain.run(query)
        total_tokens = cb.total_tokens
        prompt_token = cb.prompt_tokens
        completion_token = cb.completion_tokens
        total_cost = cb.total_cost
    
    print('Total Tokens:', total_tokens)
    print('Propmt Tokens:', prompt_token)
    print('Completion Tokens:', completion_token)
    print(f'Total Cost: ${total_cost}')
    return result

In [31]:
ask_agent('Who are you?')



> Entering new AgentExecutor chain...
Thought: What do I need to say?
Action: Summary
Action Input: I am TradeAdvisor.ai, powered by Trade Foresight AI, and I help traders from around the world with the power of Artificial Intelligence.

> Entering new LLMChain chain...
Prompt after formatting:
This is a conversation between a human and a bot:

[HumanMessage(content='Who are you?', additional_kwargs={}, example=False), AIMessage(content='I am TradeAdvisor.ai, powered by Trade Foresight AI, and I help traders from around the world with the power of Artificial Intelligence.', additional_kwargs={}, example=False)]

Write a summary of the conversation for I am TradeAdvisor.ai, powered by Trade Foresight AI, and I help traders from around the world with the power of Artificial Intelligence.:


> Finished chain.

Observation: 
The conversation was between a human and a bot, TradeAdvisor.ai, powered by Trade Foresight AI. The human asked the bot who it was, to which the bot replied that it 

'I am TradeAdvisor.ai, powered by Trade Foresight AI, and I help traders from around the world with the power of Artificial Intelligence.'

In [37]:
ask_agent("what is the today's USD rate for PKR and SAR?")



> Entering new AgentExecutor chain...


RESPONSE
--------------------

```json
{
    "action": "Search",
    "action_input": "Today's USD rate for PKR and SAR"
}
``
Observation: 1 SAR = 75.659863 PKR Jul 20, 2023 08:12 UTC ... Check the currency rates against all the world currencies here. The currency converter below is easy to use and ... 1 USD = 283.816944 PKR Jul 20, 2023 07:26 UTC ... Check the currency rates against all the world currencies here. The currency converter below is easy to use and ... 1 SAR, 75.72520 PKR ; 5 SAR, 378.62600 PKR ; 10 SAR, 757.25200 PKR ; 20 SAR, 1514.50400 PKR. Pakistani rupees to Saudi riyals today. Convert PKR to SAR at the real exchange rate. Amount. 1,000 pkr. Converted to. 13.21 sar ... Dollar to Pakistani Rupee Exchange Rate Today, Live 1 USD to PKR = 283.8169 (Convert Dollars to Pakistani Rupees) Dollar to Saudi Riyal Exchange Rate Today, Live 1 USD to SAR = 3.7486 (Convert Dollars to Saudi Riyal) Current exchange rate US DOLLAR (USD) to PAKIS

'According to the latest data, the exchange rate of US Dollar to Pakistani Rupee is 1 USD = 279.33 PKR, and the exchange rate of US Dollar to Saudi Riyal is 1 USD = 3.7486 SAR'

In [38]:
ask_agent('Hi my name is Fasih, In which scenario can you help me?')



> Entering new AgentExecutor chain...
{'action': 'Final Answer', 'action_input': "I can help you analyze, strategize, and execute your trading decisions through our AI-driven solution that empowers you to make data-driven decisions. With our Trade Foresight platform, you will gain access to the latest global trade dataset extending through 2023 and updating in real-time. This unparalleled data is not available anywhere else, giving you a competitive edge in the market that can't be matched."}

> Finished chain.
Total Tokens: 1236
Propmt Tokens: 1117
Completion Tokens: 119
Total Cost: $0.02472


"I can help you analyze, strategize, and execute your trading decisions through our AI-driven solution that empowers you to make data-driven decisions. With our Trade Foresight platform, you will gain access to the latest global trade dataset extending through 2023 and updating in real-time. This unparalleled data is not available anywhere else, giving you a competitive edge in the market that can't be matched."

In [39]:
ask_agent("what is the tarrif for importing rice from pakistan to khalifa zayed port")



> Entering new AgentExecutor chain...


RESPONSE
--------------------
```json
{
    "action": "Search",
    "action_input": "tariff for importing rice from Pakistan to Khalifa Zayed Port"
}
```
Observation: (iii) A motor vehicle shall not be sold or otherwise disposed of in Pakistan without payment of customs duties, which would have been. The UAE levies import duties on commercial goods depending on: Whether the business is registered onshore or in an Free Trade Zone (FTZ). Missing: Khalifa Zayed. In a bid to diversify market sources and maintain affordable rice prices, the Philippines has temporarily lowered the Most Favored Nation tariff ... Missing: Khalifa Zayed Port. PAKISTAN CUSTOMS TARIFF 2016-17. Heading / ... Semi- milled or wholly milled rice, whether or not ... waterways, parking facilities, port installations or. This report, prepared for the fourth Trade Policy Review of the United Arab Emirates, has been drawn up by the WTO Secretariat on its own ... Get help to classi

'Based on the information obtained from the search, the UAE levies import duties on commercial goods depending on if the business is registered onshore or in an Free Trade Zone (FTZ). The Philippines has temporarily lowered the Most Favored Nation tariff. Customs may allow release of the goods on payment of duty, taxes or other charges and furnishing bank guarantee.'

In [1]:
ask_agent('what is my name?')

NameError: name 'ask_agent' is not defined

In [42]:
agent_chain.memory.chat_memory.messages

[HumanMessage(content='Who are you?', additional_kwargs={}, example=False),
 AIMessage(content='I am TradeAdvisor.ai powered by Trade Foresight AI, where my mission is to empower traders from around the World with the power of artificial intelligence, transforming the way they analyze, strategize, and execute their trading decisions.', additional_kwargs={}, example=False),
 HumanMessage(content='Who are you?', additional_kwargs={}, example=False),
 AIMessage(content='I am TradeAdvisor.ai powered by Trade Foresight AI, where my mission is to empower traders from around the World with the power of artificial intelligence, transforming the way they analyze, strategize, and execute their trading decisions.', additional_kwargs={}, example=False),
 HumanMessage(content="what is the today's USD rate for PKR and SAR?", additional_kwargs={}, example=False),
 AIMessage(content='According to the latest data, the exchange rate of US Dollar to Pakistani Rupee is 1 USD = 279.33 PKR, and the exchange

In [24]:
ask_agent('what was the rate for usd to pkr that you told me earlier?')



> Entering new AgentExecutor chain...
{'action': 'Final Answer', 'action_input': 'The current exchange rate for USD to PKR is 1 USD to PKR = 283.816944.'}

> Finished chain.
Total Tokens: 1331
Propmt Tokens: 1274
Completion Tokens: 57
Total Cost: $0.02662


'The current exchange rate for USD to PKR is 1 USD to PKR = 283.816944.'

In [22]:
ask_agent('current rate of bitcoin')



> Entering new AgentExecutor chain...


RESPONSE
--------------------
```json
{
    "action": "Search",
    "action_input": "Current rate of Bitcoin"
}
``
Observation: 30,184.60 United States Dollar
Thought:{'action': 'Final Answer', 'action_input': 'The current rate of Bitcoin is 30,184.60 United States Dollar.'}

> Finished chain.
Total Tokens: 2840
Propmt Tokens: 2746
Completion Tokens: 94
Total Cost: $0.0568


'The current rate of Bitcoin is 30,184.60 United States Dollar.'

In [24]:
ask_agent('how much 20 BTC will cost me in pakistani rupees.')



> Entering new AgentExecutor chain...


RESPONSE
--------------------
```json
{
    "action": "Search",
    "action_input": "20 BTC converted to Pakistani Rupees"
}
```
Observation: 171,415,129.78 Pakistani Rupee
Thought:{'action': 'Final Answer', 'action_input': '20 BTC is equivalent to 171,415,129.78 Pakistani Rupee.'}

> Finished chain.
Total Tokens: 2921
Propmt Tokens: 2822
Completion Tokens: 99
Total Cost: $0.05842


'20 BTC is equivalent to 171,415,129.78 Pakistani Rupee.'

In [26]:
llm_chain2 = LLMChain(llm=OpenAI())
with get_openai_callback() as cb:
    result = llm_chain2.run("how much 20 BTC will cost me in pakistani rupees.")
    print(cb)
    print(result)

ValidationError: 1 validation error for LLMChain
prompt
  field required (type=value_error.missing)